# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

C:\Users\Admin\projects\llm_engineering\week8\agents\deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:32<00:00, 18.53s/it]


In [4]:
len(deals)

49

In [5]:
deals[44].describe()

"Title: Kobalt Tool Deals at Lowe's: Up to 50% off + free shipping w/ $35\nDetails: In this sale you'll find discounts on over 100 Kobalt tools, with prices starting from $1.98. We've pictured the Kobalt Tongue and Groove Plier Set for $14.98 (7 off). Choose in-store pickup to dodge the $5.99 shipping fee, or get free shipping with orders of $35 or more by signing up to MyLowe's Rewards. (It's free to join.) Shop Now at Lowe's\nFeatures: \nURL: https://www.dealnews.com/Kobalt-Tool-Deals-at-Lowes-Up-to-50-off-free-shipping-w-35/21715771.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Keysmart Smartcard Wallet Tracker 2-Pack for $40 + free shipping
Details: Get a two pack for $39.99 which is the best price we've seen per card. Apply coupon code "DEALNEWS" to get free shipping. Buy Now at SideDeal
Features: Works with Apple Find My App 100mAh battery
URL: https://www.dealnews.com/products/Key-Smart/Keysmart-Smartcard-Wallet-Tracker/486791.html?ire

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The Samsung Galaxy S23+ is a refurbished smartphone featuring a 256GB storage capacity. This device boasts advanced features and premium performance, powered by a One-Year Allstate warranty, offering peace of mind with your purchase. The Galaxy S23+ is a well-rounded smartphone that meets both productivity and entertainment needs.', price=490.0, url='https://www.dealnews.com/Refurb-Samsung-Deals-at-eBay-Up-to-60-off-free-shipping/21715727.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description="The Keysmart Smartcard Wallet Tracker is a compact device designed to help you keep track of your wallet by utilizing Apple's Find My App. Each tracker is powered by a rechargeable 100mAh battery, ensuring prolonged usage without frequent replacements. Offered in a convenient two-pack, this product is ideal for individuals looking for a reliable way to locate their wallets when misplaced. The pack is perfect for sharing with a partner or keeping one as a backup, adding an extra layer of security to your daily routine.", price=39.99, url='https://www.dealnews.com/products/Key-Smart/Keysmart-Smartcard-Wallet-Tracker/486791.html?iref=rss-c142'), Deal(product_description='The Certified Refurb Sony WF-1000XM5 Wireless Noise Canceling Headphones provide an immersive audio experience with their advanced noise-cancellation technology. They are equipped with ambient sound modes, allowing users to control their listening depending on their environme